In [193]:
# Import 

import arcpy
from arcpy import env
import os
from arcpy import mp


In [194]:
# Set the workspace

env.workspace = r"C:\Mac\Home\Documents\Modeling Geographical Objects\Week 4"  
env.overwriteOutput = True

In [195]:
# Define paths to shapefile and dbf
county_shapefile = r"C:\Mac\Home\Documents\Modeling Geographical Objects\Week 4\Week 04 Tutorial Data\TheCounties.shp"
county_dbf = r"C:\Mac\Home\Documents\Modeling Geographical Objects\Week 4\Week 04 Tutorial Data\CountyTable.dbf"  

# Create a feature layer from the shapefile
arcpy.MakeFeatureLayer_management(county_shapefile, "county_layer")

# Join the dbf file to the shapefile based on FIPS
arcpy.AddJoin_management("county_layer", "FIPS", county_dbf, "FIPS")

<Result 'county_layer'>

In [196]:
# select my state of interest: PA
state_name = 'Pennsylvania'
arcpy.SelectLayerByAttribute_management("county_layer", "NEW_SELECTION", "TheCounties.STATE_NAME = '{}'".format(state_name))

# Exclude counties with missing into for d and r
arcpy.SelectLayerByAttribute_management(
    "county_layer", 
    "SUBSET_SELECTION",  # This refines the existing selection
    "NOT (d = 0 AND r = 0)"
)

<Result 'county_layer'>

In [197]:
# Define the output shapefile path for the export
output_feature_class = r"C:\Mac\Home\Documents\Modeling Geographical Objects\Week 4\Exported_County_Layer.shp"

# Export the joined data to a new shapefile (keeping both geometry and the joined fields)
arcpy.FeatureClassToFeatureClass_conversion("county_layer", r"C:\Mac\Home\Documents\Modeling Geographical Objects\Week 4", "Exported_County_Layer")

# Create a feature layer from the exported shapefile
arcpy.MakeFeatureLayer_management(output_feature_class, "county_layer_V2")

# Add the Vote_Marg field
arcpy.AddField_management("county_layer_V2", "Vote_Marg", "DOUBLE")

# Calculate the vote margin using the exported fields (no more prefix issues)
arcpy.CalculateField_management(
    "county_layer_V2", 
    "Vote_Marg", 
    "abs((!d! or 0) - (!r! or 0))", 
    "PYTHON3"
)


<Result 'county_layer_V2'>

In [198]:
# classify swing and non swing counties

arcpy.AddField_management("county_layer_V2", "Swing", "SHORT")
arcpy.CalculateField_management(
    "county_layer_V2", 
    "Swing", 
    "1 if !Vote_Marg! <= 500 else 0", 
    "PYTHON3"
)

<Result 'county_layer_V2'>

In [206]:
# Add a new field for total votes (Democratic + Republican)
arcpy.AddField_management("county_layer_V2", "Tot_Vote", "DOUBLE")

# Calculate total votes
arcpy.CalculateField_management(
    "county_layer_V2", 
    "Tot_Vote", 
    "(!d! or 0) + (!r! or 0)", 
    "PYTHON3"
)

<Result 'county_layer_V2'>

In [200]:
# Add a new field for swinginess score
arcpy.AddField_management("county_layer_V2", "Swinginess", "DOUBLE")

# Calculate swinginess score
arcpy.CalculateField_management(
    "county_layer_V2", 
    "Swinginess", 
    "1 / ((abs(!d! - !r!) / (!d! + !r!)))", 
    "PYTHON3"
)

<Result 'county_layer_V2'>

In [201]:

# Reference the current map and layer
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_doc = aprx.activeMap 
county_layer = map_doc.listLayers("county_layer_V2")[0]  

# Apply graduated color symbology to the Swinginess field
sym = county_layer.symbology
sym.updateRenderer('GraduatedColorsRenderer') 
sym.renderer.classificationField = "Swinginess" 
sym.renderer.breakCount = 5  
sym.renderer.colorRamp = aprx.listColorRamps("Reds (Continuous)")[0]  

# Update the symbology of the layer
county_layer.symbology = sym